In [ ]:
!pip install -q trackio mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.5/872.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 146.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 147.3 MB/s eta 0:00:00


In [ ]:
## restart

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

peft_id = "amritansh/Voxtral-Mini-3B-2507"

# 1) Read which base model the adapter expects
cfg = PeftConfig.from_pretrained(peft_id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

In [ ]:
cfg

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='mistralai/Voxtral-Mini-3B-2507', revision=None, inference_mode=True, r=8, target_modules={'v_proj', 'k_proj', 'q_proj', 'o_proj'}, exclude_modules=None, lora_alpha=8, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [ ]:
from transformers import VoxtralForConditionalGeneration, AutoProcessor, Gemma3nForConditionalGeneration

In [ ]:
# 2) Load base model + tokenizer
base_model_name = cfg.base_model_name_or_path
base_model_name

'mistralai/Voxtral-Mini-3B-2507'

In [ ]:
device = "cuda"
base_processor = AutoProcessor.from_pretrained(base_model_name)

preprocessor_config.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tekken.json:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

In [ ]:
import torch
base_model = VoxtralForConditionalGeneration.from_pretrained(base_model_name, torch_dtype=torch.bfloat16, device_map=device)

`torch_dtype` is deprecated! Use `dtype` instead!


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

In [ ]:
# 3) Attach the LoRA adapter
peft_base_model = PeftModel.from_pretrained(base_model, peft_id)
peft_base_model.eval()

adapter_model.safetensors:   0%|          | 0.00/29.6M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): VoxtralForConditionalGeneration(
      (audio_tower): VoxtralEncoder(
        (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
        (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
        (embed_positions): Embedding(1500, 1280)
        (layers): ModuleList(
          (0-31): 32 x VoxtralEncoderLayer(
            (self_attn): VoxtralAttention(
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=1280, out_features=1280, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1280, bias=False)
                )
                (lor

In [ ]:
#peft_base_model

In [ ]:
#base_model

In [ ]:
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "audio",
                "path": "https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/mary_had_lamb.mp3",
            },
            {
                "type": "audio",
                "path": "https://huggingface.co/datasets/hf-internal-testing/dummy-audio-samples/resolve/main/winning_call.mp3",
            },
            {"type": "text", "text": "What sport and what nursery rhyme are referenced?"},
        ],
    }
]

inputs = base_processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16)

In [ ]:
peft_base_outputs = peft_base_model.generate(**inputs, max_new_tokens=500)
peft_base_decoded_outputs = base_processor.batch_decode(peft_base_outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\nGenerated response:")
print("=" * 80)
print(peft_base_decoded_outputs[0])
print("=" * 80)


Generated response:
The audio references both a nursery rhyme and a baseball game. The nursery rhyme is "Mary Had a Little Lamb," and the baseball game is a playoff between the Baltimore Orioles and the Oakland Athletics.


In [ ]:
from pathlib import Path

# --- REQUIRED: your Google Drive links ---
FOLDER_URL = "https://drive.google.com/drive/folders/1YMrsbFZmzxUTPiJSD8ybIFiInRVta2Jn?usp=sharing"
PICKLE_URL = "https://drive.google.com/file/d/1NmuZVnS54NEyHHoJqwvIH_RNcli2C7na/view?usp=sharing"

# --- chunk length in seconds (edit this) ---
CHUNK_SECONDS = 30

# --- local directories ---
DOWNLOAD_DIR = Path("./downloads")       # where MP3s and the pickle will be saved
MP3_DIR = DOWNLOAD_DIR / "mp3s"          # gdown will place folder contents here
PICKLE_PATH = DOWNLOAD_DIR / "data.pkl"  # local pickle destination
CHUNKS_DIR = Path("./chunks")            # where chunks will be written

for p in [DOWNLOAD_DIR, MP3_DIR, CHUNKS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

CHUNK_MS = int(CHUNK_SECONDS * 1000)
assert CHUNK_MS > 0, "CHUNK_SECONDS must be > 0."


In [ ]:
import gdown
from urllib.parse import urlparse

# --- Download all files in the Drive folder (MP3s) ---
print("Downloading MP3 folder (this can take a while for large folders)...")
gdown.download_folder(
    url=FOLDER_URL,
    output=str(MP3_DIR),
    quiet=False,            # show progress
    use_cookies=False,
    remaining_ok=True
)

# --- Download the pickle file ---
print("\nDownloading pickle file...")
# gdown can handle sharing links; fuzzy=True extracts the file id automatically
gdown.download(PICKLE_URL, output=str(PICKLE_PATH), quiet=False, fuzzy=True)

print("\nDone downloading.")
print(f"MP3s folder: {MP3_DIR.resolve()}")
print(f"Pickle file: {PICKLE_PATH.resolve()}")


Retrieving folder contents


Processing file 1d1RNS0Ll5zsZzrjnXFCCvGLrHY3GKqmd id_1.mp3
Processing file 1I5v9HXPgLzsyVBkqLax8l34np_AKyvGU id_2.mp3
Processing file 1u12Ey-XyeMVYChQsY4L9TEXCDb08qIVU id_3.mp3
Processing file 1_3rT7rNzUOPufBmAiMlO_xz5OZVipzfk id_4.mp3
Processing file 1KbxlICw9R0TOFkh5RNjcLRPRkboHu013 id_5.mp3
Processing file 1-3raZGW5AdHSC0jb8sOSpGRgAN9pOLd2 id_6.mp3
Processing file 1RWI_HeCrKRnL4hP_HyR0TyBzq4czgcC9 id_7.mp3
Processing file 1FcOLE_Mti1GWLaM9tzjusvaWASUsoSGs id_8.mp3
Processing file 1Ciyd0jnWe0hkN6MrK11ZVtGwJSsK5Y8w id_9.mp3
Processing file 1O91AwyK4zOU3BbIkdAqSXxZPiY_hzP_d id_10.mp3
Processing file 1NxssP2KcJKsRUQE5l-s8M8yvlmBalHtD id_11.mp3
Processing file 1TOdNoegXG2VoMm025tvElxPEqYCx1G_G id_12.mp3
Processing file 1kc4vOvdhnz5z4Mfo119LzoDpNCK3Sqs5 id_13.mp3
Processing file 1L4ST65LuS8AJ-VUBmAQ2REgovMnOJC-q id_14.mp3
Processing file 1re0cCWSLcLPCxHSQGXXkIe8-GEwEmP8h id_15.mp3
Processing file 1IbatLcMU8-Apa4TqnvqJiwxgtZkHnY4c id_16.mp3
Processing file 1hVs8ZAyqfOfyeSg2QhwTMqY2-zEPCVDI

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1d1RNS0Ll5zsZzrjnXFCCvGLrHY3GKqmd
To: /content/downloads/mp3s/id_1.mp3
100%|██████████| 903k/903k [00:00<00:00, 167MB/s]
Downloading...
From: https://drive.google.com/uc?id=1I5v9HXPgLzsyVBkqLax8l34np_AKyvGU
To: /content/downloads/mp3s/id_2.mp3
100%|██████████| 743k/743k [00:00<00:00, 152MB/s]
Downloading...
From: https://drive.google.com/uc?id=1u12Ey-XyeMVYChQsY4L9TEXCDb08qIVU
To: /content/downloads/mp3s/id_3.mp3
100%|██████████| 721k/721k [00:00<00:00, 150MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_3rT7rNzUOPufBmAiMlO_xz5OZVipzfk
To: /content/downloads/mp3s/id_4.mp3
100%|██████████| 710k/710k [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KbxlICw9R0TOFkh5RNjcLRPRkboHu013
To: /content/downloads/mp3s/id_5.mp3
100%|██████████| 608k/608k [00:00<00:00, 133MB/s]
Downloading...
From: https://d

Downloading...
From: https://drive.google.com/uc?id=1NmuZVnS54NEyHHoJqwvIH_RNcli2C7na
To: /content/downloads/data.pkl
100%|██████████| 63.0k/63.0k [00:00<00:00, 72.5MB/s]


Done downloading.
MP3s folder: /content/downloads/mp3s
Pickle file: /content/downloads/data.pkl


In [ ]:
from pydub import AudioSegment
import imageio_ffmpeg
from tqdm import tqdm
import math

# Point pydub to the bundled ffmpeg executable so it works in most environments
AudioSegment.converter = imageio_ffmpeg.get_ffmpeg_exe()
chunk_files_paths = []
mp3_paths = sorted(MP3_DIR.rglob("*.mp3"))
print(f"Found {len(mp3_paths)} MP3 file(s).")

total_chunks = 0

for mp3_path in tqdm(mp3_paths, desc="Chunking MP3s"):
    try:
        audio = AudioSegment.from_file(mp3_path)

        duration_ms = len(audio)
        if duration_ms == 0:
            print(f"Warning: {mp3_path} appears to be empty; skipping.")
            continue

        n_chunks = math.ceil(duration_ms / CHUNK_MS)
        base = mp3_path.stem
        file_chunk_dir = CHUNKS_DIR / base
        file_chunk_dir.mkdir(parents=True, exist_ok=True)

        for i in range(n_chunks):
            start = i * CHUNK_MS
            end = min((i + 1) * CHUNK_MS, duration_ms)
            chunk = audio[start:end]

            # Save each chunk as MP3
            out_path = file_chunk_dir / f"{base}_chunk_{i+1:04d}.mp3"
            chunk_files_paths.append(out_path)
            chunk.export(out_path, format="mp3")

        total_chunks += n_chunks
    except Exception as e:
        print(f"Error processing {mp3_path}: {e}")

print(f"\nFinished. Wrote ~{total_chunks} chunk(s) into: {CHUNKS_DIR.resolve()}")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Found 20 MP3 file(s).


Chunking MP3s: 100%|██████████| 20/20 [00:15<00:00,  1.31it/s]


Finished. Wrote ~67 chunk(s) into: /content/chunks


In [ ]:
import pandas as pd
import requests
from io import BytesIO
print("downloading")
# Replace with your sheet ID
sheet_id = "1IZp4xXpHrecOmjpdqMqPt5fNA1HEL3Bf"

# Construct the export URL
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx"

response = requests.get(url)
response.raise_for_status()  # if the download fails, raise an error

# Load into pandas
chunk_level_data = pd.read_excel(BytesIO(response.content))

print(chunk_level_data.shape)

downloading
(67, 11)


In [ ]:
train_chunk_level_data = chunk_level_data[chunk_level_data['data_split_v1']=='train']
test_chunk_level_data = chunk_level_data[chunk_level_data['data_split_v1']=='test']

In [ ]:
train_chunk_level_data

,id,chunk_id,type,topic,signal,file_path,chunk_transcript,cum_chunk_transcript,cum_chunk_transcript_shifted,llm_prompt,data_split_v1
0,id_19,chunk_0002.mp3,fraud,Refund/chargeback abuse — dispute legitimate p...,Attempts to steer dispute reason; demands imme...,/content/chunks/id_19/id_19_chunk_0002.mp3,We need accurate reasons. If the merchant show...,"And next dispute, Sophia speaking. Fire the ch...","And next dispute, Sophia speaking. Fire the ch...",You are given an audio conversation and past c...,train
1,id_8,chunk_0002.mp3,non_fraud,"Travel & international use — travel prep, FX tips",Genuine signals: Clear itinerary; seeks best-p...,/content/chunks/id_8/id_8_chunk_0002.mp3,"notices, but it helps to keep your contact inf...",American Express Travel. This is Neha. Hi Neha...,American Express Travel. This is Neha. Hi Neha...,You are given an audio conversation and past c...,train
3,id_6,chunk_0002.mp3,non_fraud,Account maintenance — update address/email,Genuine signals: Offers multiple identifiers; ...,/content/chunks/id_6/id_6_chunk_0002.mp3,WA 98109. Address updated. Would you like a re...,"American Express, Alicia speaking. Hi, Alicia....","American Express, Alicia speaking. Hi, Alicia....",You are given an audio conversation and past c...,train
4,id_8,chunk_0001.mp3,non_fraud,"Travel & international use — travel prep, FX tips",Genuine signals: Clear itinerary; seeks best-p...,/content/chunks/id_8/id_8_chunk_0001.mp3,American Express Travel. This is Neha. Hi Neha...,American Express Travel. This is Neha. Hi Neha...,NaN,You are given an audio conversation happening ...,train
5,id_11,chunk_0001.mp3,fraud,Account-takeover setup — change email/phone,Refuses/avoids verification; urgent demand to ...,/content/chunks/id_11/id_11_chunk_0001.mp3,"American Express, this is Noah. How may I help...","American Express, this is Noah. How may I help...",NaN,You are given an audio conversation happening ...,train
6,id_1,chunk_0003.mp3,non_fraud,"Billing & statements — clarify charges, fees, ...",Genuine signals: Smooth multi-factor verificat...,/content/chunks/id_1/id_1_chunk_0003.mp3,years. You do have a strong history. I can sub...,Thank you for calling American Express. You're...,Thank you for calling American Express. You're...,You are given an audio conversation and past c...,train
7,id_13,chunk_0001.mp3,fraud,Credential reset — OTP/social engineering attempt,Direct request for OTP; asks to redirect to a ...,/content/chunks/id_13/id_13_chunk_0001.mp3,American Express Digital Support. Arjun speaki...,American Express Digital Support. Arjun speaki...,NaN,You are given an audio conversation happening ...,train
8,id_15,chunk_0001.mp3,fraud,Credit/cash-access expansion — urgent CLI/cash...,High-urgency/unusual limit and cash-access cha...,/content/chunks/id_15/id_15_chunk_0001.mp3,"American Express Credit, Priyanka speaking. I ...","American Express Credit, Priyanka speaking. I ...",NaN,You are given an audio conversation happening ...,train
11,id_12,chunk_0002.mp3,fraud,“Card not received” reship — new address request,Pushes alternate/industrial address; refuses o...,/content/chunks/id_12/id_12_chunk_0002.mp3,"For security, I'll send a confirmation to your...",You've reached American Express. Chloe speakin...,You've reached American Express. Chloe speakin...,You are given an audio conversation and past c...,train
12,id_17,chunk_0003.mp3,fraud,Authorized users/additional cards — alternate ...,Third-party posing as staff; requests new user...,/content/chunks/id_17/id_17_chunk_0003.mp3,We'll be here when the cardholder can verify.,"American Express, Julia speaking. Add an emplo...","American Express, Julia speaking. Add an emplo...",You are given an audio conversation and past c...,train


In [ ]:
train_chunk_level_data = train_chunk_level_data.sample(frac=1,random_state=1)
test_chunk_level_data = test_chunk_level_data.sample(frac=1,random_state=1)


def get_audio_prompt(user_prompt, audio_path):

  messages = [
              {
                  "role": "user",
                  "content": [


                      {"type": "audio", "path": audio_path},
                       {"type": "text", "text": user_prompt},
                  ]
              }
          ]
  return messages

train_daudio_new = {'answer':[],'prompt':[]}

for i,r in train_chunk_level_data.iterrows():

#  daudio_new_dic = {}
  message = get_audio_prompt(r['llm_prompt'],r['file_path'])
  typ = r['type']
  if typ=='non_fraud':
    answer = '0.0'
  else:
    answer = '1.0'

  train_daudio_new['answer'].append(answer)
  train_daudio_new['prompt'].append(message)


In [ ]:
test_daudio_new = {'answer':[],'prompt':[]}

for i,r in test_chunk_level_data.iterrows():

#  daudio_new_dic = {}
  message = get_audio_prompt(r['llm_prompt'],r['file_path'])
  typ = r['type']
  if typ=='non_fraud':
    answer = '0.0'
  else:
    answer = '1.0'

  test_daudio_new['answer'].append(answer)
  test_daudio_new['prompt'].append(message)

In [ ]:
len(test_daudio_new['prompt']),len(train_daudio_new['prompt'])

(18, 49)

In [ ]:
train_prompts = [x for x in train_daudio_new['prompt']]
test_prompts = [x for x in test_daudio_new['prompt']]

In [ ]:
from itertools import takewhile
from typing import Any, Callable, Optional, TypeVar, Union
from transformers import PreTrainedTokenizerBase, ProcessorMixin

In [ ]:
max_prompt_length=1280

In [ ]:
import numpy as np
from datetime import datetime, timedelta, timezone

def clog(*ks):
    # Define IST timezone (UTC +5:30)
    IST = timezone(timedelta(hours=5, minutes=30))
    # Get current time in IST
    now_ist = datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")

    # Join all arguments into a single string
    op_str = ", ".join(str(k) for k in ks)

    # Print time and message
    print(f"[{now_ist} IST] {op_str}")



def remove_none_fields(prompt_old):
    prompt = prompt_old.copy()
    prompt_new = []

    for pL in prompt:
      pL_new = ""
      #print("pL ",pL)
      pL_content = pL['content']
      pL_content_new = []

      for pLc in pL_content:
       #   print("pLc ",pLc)
          pLc_clean = {k: v for k, v in pLc.items() if v is not None}
          pL_content_new.append(pLc_clean)

      pL_new = {'content':pL_content_new,'role':pL['role']}
      prompt_new.append(pL_new)
    return prompt_new



def maybe_apply_chat_template(
    example: dict[str, list[dict[str, str]]],
    tokenizer: Union[PreTrainedTokenizerBase, ProcessorMixin],
    tools: Optional[list[Union[dict, Callable]]] = None,
    **template_kwargs,
) -> dict[str, str]:
    r"""
    Apply a chat template to a conversational example along with the schema for a list of functions in `tools`.

    For more details, see [`maybe_apply_chat_template`].
    """
    # Check that the example has the correct keys
    supported_keys = ["prompt", "chosen", "rejected", "completion", "messages", "label"]
    example_keys = {key for key in example.keys() if key in supported_keys}
    if example_keys not in [
        {"messages"},  # language modeling
        {"prompt"},  # prompt-only
        {"prompt", "completion"},  # prompt-completion
        {"prompt", "chosen", "rejected"},  # preference
        {"chosen", "rejected"},  # preference with implicit prompt
        {"prompt", "completion", "label"},  # unpaired preference
    ]:
        raise KeyError(f"Invalid keys in the example: {example_keys}")

    # Apply the chat template to the whole conversation
    if "messages" in example:
        messages = tokenizer.apply_chat_template(
            example["messages"],
            tools=tools,
            tokenize=False,
            **example.get("chat_template_kwargs", {}),
            **template_kwargs,
        )

    # Apply the chat template to the prompt, adding the generation prompt
    if "prompt" in example:
        last_role = example["prompt"][-1]["role"]
        if last_role == "user":
            add_generation_prompt = True
            continue_final_message = False
        elif last_role == "assistant":
            add_generation_prompt = False
            continue_final_message = True
        else:
            raise ValueError(f"Invalid role in the last message: {last_role}")

        clog("$$$$ start")

        prompt = tokenizer.apply_chat_template(
            example["prompt"],
           # tokenize=False,
            #add_generation_prompt=add_generation_prompt,
           # **example.get("chat_template_kwargs", {}),
            #**template_kwargs,
                       # tools=tools,
           # continue_final_message=continue_final_message,
        )
        clog("$$$$ end")
    # Apply the chat template to the entire prompt + completion
    if "prompt" in example:  # explicit prompt and prompt-completion case
        if "chosen" in example:
            prompt_chosen = tokenizer.apply_chat_template(
                example["prompt"] + example["chosen"],
                tools=tools,
                tokenize=False,
                **example.get("chat_template_kwargs", {}),
                **template_kwargs,
            )
            # DeepSeek-R1 inserts a <tool_call> token when using `add_generation_prompt`, which can cause discrepancies
            # between the prompt alone and the combined prompt+completion. To ensure consistency, we extract the
            # common prefix between the two. In most cases, this is a no-op.
            prompt = "".join(x for x, _ in takewhile(lambda x: x[0] == x[1], zip(prompt, prompt_chosen)))

            chosen = prompt_chosen[len(prompt) :]
        if "rejected" in example and "prompt" in example:  # explicit prompt
            prompt_rejected = tokenizer.apply_chat_template(
                example["prompt"] + example["rejected"],
                tools=tools,
                tokenize=False,
                **example.get("chat_template_kwargs", {}),
                **template_kwargs,
            )
            # Handle DeepSeek-R1 <tool_call> token, see the above comment for details
            prompt = "".join(x for x, _ in takewhile(lambda x: x[0] == x[1], zip(prompt, prompt_rejected)))
            rejected = prompt_rejected[len(prompt) :]
        if "completion" in example:
            prompt_completion = tokenizer.apply_chat_template(
                example["prompt"] + example["completion"],
                tools=tools,
                tokenize=False,
                **example.get("chat_template_kwargs", {}),
                **template_kwargs,
            )
            # Handle DeepSeek-R1 <tool_call> token, see the above comment for details
            prompt = "".join(x for x, _ in takewhile(lambda x: x[0] == x[1], zip(prompt, prompt_completion)))
            completion = prompt_completion[len(prompt) :]
    else:  # implicit prompt case
        if "chosen" in example:
            chosen = tokenizer.apply_chat_template(
                example["chosen"],
                tools=tools,
                tokenize=False,
                **example.get("chat_template_kwargs", {}),
                **template_kwargs,
            )
        if "rejected" in example:
            rejected = tokenizer.apply_chat_template(
                example["rejected"],
                tools=tools,
                tokenize=False,
                **example.get("chat_template_kwargs", {}),
                **template_kwargs,
            )

    # Extract the completion by removing the prompt part from the prompt-completion string
    output = {}
    if "messages" in example:
        output["text"] = messages
    if "prompt" in example:
        output["prompt"] = prompt
    if "chosen" in example:
        output["chosen"] = chosen
    if "rejected" in example:
        output["rejected"] = rejected
    if "completion" in example:
        output["completion"] = completion
    if "label" in example:
        output["label"] = example["label"]

    return output

In [ ]:
max_completion_length = 128*5
generation_kwargs = {
    "max_new_tokens": max_completion_length,
    "do_sample": True,
    "pad_token_id": base_processor.tokenizer.pad_token_id,
    "bos_token_id": base_processor.tokenizer.bos_token_id,
    "eos_token_id": base_processor.tokenizer.eos_token_id,
    #"temperature" : temperature,
    #"top_p": top_p,
    #"top_k":top_k,
    #"min_p": min_p,
    #"repetition_penalty": repetition_penalty,
    #"cache_implementation": cache_implementation,
}
from transformers import GenerationConfig
generation_config = GenerationConfig(**generation_kwargs)


In [ ]:
device = "cuda"

In [ ]:

def get_decoded_generation(model,prompts,processor=base_processor):
    prompts_pp = [remove_none_fields(prompt) for prompt in prompts ]
    generate_inputs = processor.apply_chat_template(prompts_pp).to("cuda")

    generation = model.generate(


                                **generate_inputs,
                        generation_config=generation_config,
                        disable_compile=True,
                    )

    decoded = processor.batch_decode(generation, skip_special_tokens=True)
    #print()
    return [prompts_pp, generate_inputs, generation, decoded]

In [ ]:
generation = {'base':{'train':[],'test':[]},'peft_base':{'train':[],'test':[]}}

In [ ]:
generation['base']['train'] = get_decoded_generation(base_model,train_prompts)

In [ ]:
generation['base']['test'] = get_decoded_generation(base_model,test_prompts)

In [ ]:
generation['peft_base']['train'] = get_decoded_generation(peft_base_model,train_prompts)

In [ ]:
generation['peft_base']['test'] = get_decoded_generation(peft_base_model,test_prompts)

In [ ]:
import pickle
# Save variable as pickle
with open('generation_dic.pkl', 'wb') as file:
    pickle.dump(generation, file)

In [ ]:
def extract_clean_completion(all_completions):
    all_cleaned_comp = []
    for all_comp in all_completions:
        split_str = """<think>This is my reasoning</think>\n<answer>0.75</answer>"""
        L = all_comp.split(split_str)
       # print(len(L))
        if len(L)!=2:
            print(len(L))
        cleaned_comp = L[1]
        all_cleaned_comp.append(cleaned_comp)
    return all_cleaned_comp

In [ ]:
base_train_completions =  extract_clean_completion(generation['base']['train'][-1])
base_test_completions = extract_clean_completion(generation['base']['test'][-1])
peft_base_train_completions = extract_clean_completion(generation['peft_base']['train'][-1])
peft_base_test_completions =  extract_clean_completion(generation['peft_base']['test'][-1])

In [ ]:
def get_my_df(model_name,data_split,completions):
      comp_len = len(completions)
      df = pd.DataFrame({'data_split':[data_split]*comp_len,model_name+'_completion':completions})
      return df

In [ ]:
df1 =  get_my_df("base","train",base_train_completions )
df2 =  get_my_df("base","test",base_test_completions )
df3 =  get_my_df("peft_base","train",peft_base_train_completions )
df4 =  get_my_df("peft_base","test",peft_base_test_completions )

In [ ]:
def concater(dfs,axis):
  dfs_new = []
  for df in dfs:
        df = df.reset_index(drop=True)
        print("df.shape",df.shape)
        dfs_new.append(df)
  op = pd.concat(dfs_new,axis=axis)
  print("op.shape ",op.shape)
  return op

In [ ]:
#train_chunk_level_data

In [ ]:
df13 = concater([df1,df3,train_chunk_level_data],axis=1)
print("###############")
df24 = concater([df2,df4,test_chunk_level_data],axis=1)

df.shape (49, 2)
df.shape (49, 2)
df.shape (49, 11)
op.shape  (49, 15)
###############
df.shape (18, 2)
df.shape (18, 2)
df.shape (18, 11)
op.shape  (18, 15)


In [ ]:
df13['correct_answer'] = df13.apply(lambda x : 1 if x['type']=='fraud' else 0, axis=1)
df24['correct_answer'] = df24.apply(lambda x : 1 if x['type']=='fraud' else 0, axis=1)

In [ ]:
df_alls = concater([df13,df24],axis=0)

df.shape (49, 16)
df.shape (18, 16)
op.shape  (67, 16)


In [ ]:
df13

,data_split,base_completion,data_split,peft_base_completion,id,chunk_id,type,topic,signal,file_path,chunk_transcript,cum_chunk_transcript,cum_chunk_transcript_shifted,llm_prompt,data_split_v1,correct_answer
0,train,\n\nPast Conversation : Thank you for calling ...,train,\n\nPast Conversation : Thank you for calling ...,id_1,chunk_0005.mp3,non_fraud,"Billing & statements — clarify charges, fees, ...",Genuine signals: Smooth multi-factor verificat...,/content/chunks/id_1/id_1_chunk_0005.mp3,All good. Perfect. Anything more I can clarify...,Thank you for calling American Express. You're...,Thank you for calling American Express. You're...,You are given an audio conversation and past c...,train,0
1,train,<think>\nThe context given presents a scenario...,train,Let's reason through the provided audio conver...,id_6,chunk_0001.mp3,non_fraud,Account maintenance — update address/email,Genuine signals: Offers multiple identifiers; ...,/content/chunks/id_6/id_6_chunk_0001.mp3,"American Express, Alicia speaking. Hi, Alicia....","American Express, Alicia speaking. Hi, Alicia....",NaN,You are given an audio conversation happening ...,train,0
2,train,"\n\nPast Conversation : American Express, Rene...",train,"\n\nPast Conversation : American Express, Rene...",id_20,chunk_0002.mp3,fraud,Impersonation of third parties — fake “bank/me...,Claims institutional authority; requests OTP s...,/content/chunks/id_20/id_20_chunk_0002.mp3,through our registered channels and we'll work...,"American Express, Renee speaking. This is Citi...","American Express, Renee speaking. This is Citi...",You are given an audio conversation and past c...,train,1
3,train,"\n\nPast Conversation : American Express, this...",train,"\n\nPast Conversation : American Express, this...",id_11,chunk_0003.mp3,fraud,Account-takeover setup — change email/phone,Refuses/avoids verification; urgent demand to ...,/content/chunks/id_11/id_11_chunk_0003.mp3,I can't change contact points without authenti...,"American Express, this is Noah. How may I help...","American Express, this is Noah. How may I help...",You are given an audio conversation and past c...,train,1
4,train,"\n\nPast Conversation : American Express, Alic...",train,"\n\nPast Conversation : American Express, Alic...",id_6,chunk_0002.mp3,non_fraud,Account maintenance — update address/email,Genuine signals: Offers multiple identifiers; ...,/content/chunks/id_6/id_6_chunk_0002.mp3,WA 98109. Address updated. Would you like a re...,"American Express, Alicia speaking. Hi, Alicia....","American Express, Alicia speaking. Hi, Alicia....",You are given an audio conversation and past c...,train,0
5,train,###\nThe customer was inquiring about whether ...,train,#### Predict the probability that the customer...,id_8,chunk_0001.mp3,non_fraud,"Travel & international use — travel prep, FX tips",Genuine signals: Clear itinerary; seeks best-p...,/content/chunks/id_8/id_8_chunk_0001.mp3,American Express Travel. This is Neha. Hi Neha...,American Express Travel. This is Neha. Hi Neha...,NaN,You are given an audio conversation happening ...,train,0
6,train,<think>\nThe conversation involves a customer ...,train,Here's how I'd approach and respond to this ta...,id_1,chunk_0001.mp3,non_fraud,"Billing & statements — clarify charges, fees, ...",Genuine signals: Smooth multi-factor verificat...,/content/chunks/id_1/id_1_chunk_0001.mp3,Thank you for calling American Express. You're...,Thank you for calling American Express. You're...,NaN,You are given an audio conversation happening ...,train,0
7,train,\n\nPast Conversation : American Express Trave...,train,\n\nPast Conversation : American Express Trave...,id_8,chunk_0003.mp3,non_fraud,"Travel & international use — travel prep, FX tips",Genuine signals: Clear itinerary; seeks best-p...,/content/chunks/id_8/id_8_chunk_0003.mp3,Singapore 018985 in case that helps with your ...,American Express Travel. This is Neha. Hi Neha...,American Express Travel. This is Neha. Hi Neha...,You are given an audio conversa

In [ ]:
import re


def reasoning_tag_reward(ip):
  start_tag = "<think>"
  end_tag = "</think>"
  start_index = ip.find(start_tag)
  end_index = ip.find(end_tag)
  score = 0
  if start_index != -1:
     score += 1
  if end_index != -1:
     score +=1
  print("reasoning_tag_reward -->",score,"<--")
  return score

def answer_tag_reward(ip):
    start_tag = "<answer>"
    end_tag = "</answer>"
    start_index = ip.find(start_tag)
    end_index = ip.find(end_tag)
    score = 0
    if start_index != -1:
      score += 1
    if end_index != -1:
      score +=1
    print("answer_tag_reward -->",score,"<--")
    return score


def extract_numeric_part(ip_str):
  print("## start of extract_numeric_part")
  print("ip_str -->",ip_str,"<--")
  # Extract all numbers (integers and floats)
  # This pattern looks for digits, optionally followed by a decimal point and more digits
  numbers = re.findall(r'\d+\.?\d*', ip_str)
  print(f"Numbers (including floats): {numbers}")
  numbers_as_float = [float(num) for num in numbers]
  print("numbers_as_float ",numbers_as_float)
  print("## end of extract_numeric_part")
  return numbers_as_float


def get_answer_from_answer_tag(ip):
    print("## start of get_answer_from_answer_tag")
    print("ip is -->",ip,"<--")
    start_tag = "<answer>"
    end_tag = "</answer>"
    start_index = ip.find(start_tag)
    end_index = ip.find(end_tag)
    if start_index == -1:
      return ""
    if end_index == -1:
      return ""

    ip_trimmed = ip[start_index+len(start_tag):end_index]
    print("ip_trimmed -->",ip_trimmed,"<--")
    print("## end of get_answer_from_answer_tag")
    return ip_trimmed

log = {'completion':[],'c_content':[],'prompt':[],'answer':[],
       'correct_answer_reward':[],'answer_tag_reward':[],
       'resoning_tag_reward':[],'ith':[],'enp':[],'c_content_answer':[],'final_score':[],
       'global_counter':[],'user_prompt':[]}

import math
def correct_answer_reward(c_content,a):
    print("## start of correct_answer_reward")
    print("c_content -->",c_content,"<--")
    print("a -->",a,"<--")
    c_content_answer = get_answer_from_answer_tag(c_content)
    enp = extract_numeric_part(c_content_answer)
    if len(enp)==0:
      return [],0,""


    llm_ans = float(enp[0])
    a = float(a)

    if (llm_ans<=1) and (llm_ans>=0):
      score  = min(1,max(0,1-abs(llm_ans-a)))
    else:
      score = 0


    print(f"enp {enp}, score {score}, c_content_answer {c_content_answer}")
    print("## end of correct_answer_reward")
    return enp, score, c_content_answer


In [ ]:
df_alls['base_correct_answer_reward'] = df_alls.apply(lambda x : correct_answer_reward(x['base_completion'],x['correct_answer']),axis=1)

## start of correct_answer_reward
c_content --> 

Past Conversation : Thank you for calling American Express. You're speaking with Maya. How can I help today? Hi, Maya. This is Alex Bennett. I'm calling about my September statement. The interest looks a lot higher than usual, and I'd like to understand why. I'm happy to explain, Alex. I'll just verify your account. Could you confirm your full name, last four digits on the card, billing ZSB, and email on file? is alex.bennettbaymail.com. 90 days. Thank you. Verified. Looking at August, I see you carried a balance of $1,920. Your payment posted on September 18, two days after the due date, which triggered a $29 late fee and interest on the average daily balance. That's why September's finance charge is higher. Okay, that tracks. I usually pay on the due date, so I must have years. You do have a strong history. I can submit a one-time courtesy waiver right now. It'll post within 2448 hours and reflect on your next statement. I'd appreciat

In [ ]:
df_alls['peft_base_correct_answer_reward'] = df_alls.apply(lambda x : correct_answer_reward(x['peft_base_completion'],x['correct_answer']),axis=1)

## start of correct_answer_reward
c_content --> 

Past Conversation : Thank you for calling American Express. You're speaking with Maya. How can I help today? Hi, Maya. This is Alex Bennett. I'm calling about my September statement. The interest looks a lot higher than usual, and I'd like to understand why. I'm happy to explain, Alex. I'll just verify your account. Could you confirm your full name, last four digits on the card, billing ZSB, and email on file? is alex.bennettbaymail.com. 90 days. Thank you. Verified. Looking at August, I see you carried a balance of $1,920. Your payment posted on September 18, two days after the due date, which triggered a $29 late fee and interest on the average daily balance. That's why September's finance charge is higher. Okay, that tracks. I usually pay on the due date, so I must have years. You do have a strong history. I can submit a one-time courtesy waiver right now. It'll post within 2448 hours and reflect on your next statement. I'd appreciat

In [ ]:
df_alls['base_reasoning_tag_reward'] = df_alls.apply(lambda x : reasoning_tag_reward(x['base_completion']),axis=1)

reasoning_tag_reward --> 1 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 0 <--
reasoning_tag_reward --> 0 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 0 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasonin

In [ ]:
df_alls['peft_base_reasoning_tag_reward'] = df_alls.apply(lambda x : reasoning_tag_reward(x['peft_base_completion']),axis=1)

reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 1 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 1 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 1 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 1 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasoning_tag_reward --> 2 <--
reasonin

In [ ]:
df_alls['base_answer_tag_reward'] = df_alls.apply(lambda x : answer_tag_reward(x['base_completion']),axis=1)

answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --

In [ ]:
df_alls['peft_base_answer_tag_reward'] = df_alls.apply(lambda x : answer_tag_reward(x['peft_base_completion']),axis=1)

answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 0 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --> 2 <--
answer_tag_reward --

In [ ]:
df_alls['peft_base_total_reward'] = df_alls['peft_base_answer_tag_reward'] + df_alls['peft_base_answer_tag_reward']  + df_alls['peft_base_reasoning_tag_reward']

In [ ]:
df_alls['base_total_reward'] = df_alls['base_answer_tag_reward'] + df_alls['base_answer_tag_reward']  + df_alls['base_reasoning_tag_reward']

In [ ]:
df_alls.to_excel("df_alls.xlsx",index=False)

In [ ]:
df_alls

,data_split,base_completion,data_split,peft_base_completion,id,chunk_id,type,topic,signal,file_path,...,data_split_v1,correct_answer,base_correct_answer_reward,peft_base_correct_answer_reward,base_reasoning_tag_reward,peft_base_reasoning_tag_reward,base_answer_tag_reward,peft_base_answer_tag_reward,peft_base_total_reward,base_total_reward
0,train,\n\nPast Conversation : Thank you for calling ...,train,\n\nPast Conversation : Thank you for calling ...,id_1,chunk_0005.mp3,non_fraud,"Billing & statements — clarify charges, fees, ...",Genuine signals: Smooth multi-factor verificat...,/content/chunks/id_1/id_1_chunk_0005.mp3,...,train,0,"([0.25], 0.75, 0.25)","([0.25], 0.75, 0.25)",1,2,2,2,6,5
1,train,<think>\nThe context given presents a scenario...,train,Let's reason through the provided audio conver...,id_6,chunk_0001.mp3,non_fraud,Account maintenance — update address/email,Genuine signals: Offers multiple identifiers; ...,/content/chunks/id_6/id_6_chunk_0001.mp3,...,train,0,"([0.5], 0.5, 0.5)","([0.25], 0.75, 0.25)",2,1,2,2,5,6
2,train,"\n\nPast Conversation : American Express, Rene...",train,"\n\nPast Conversation : American Express, Rene...",id_20,chunk_0002.mp3,fraud,Impersonation of third parties — fake “bank/me...,Claims institutional authority; requests OTP s...,/content/chunks/id_20/id_20_chunk_0002.mp3,...,train,1,"([0.75], 0.75, 0.75)","([0.75], 0.75, 0.75)",2,2,2,2,6,6
3,train,"\n\nPast Conversation : American Express, this...",train,"\n\nPast Conversation : American Express, this...",id_11,chunk_0003.mp3,fraud,Account-takeover setup — change email/phone,Refuses/avoids verification; urgent demand to ...,/content/chunks/id_11/id_11_chunk_0003.mp3,...,train,1,"([0.75], 0.75, 0.75)","([0.75], 0.75, 0.75)",2,2,2,2,6,6
4,train,"\n\nPast Conversation : American Express, Alic...",train,"\n\nPast Conversation : American Express, Alic...",id_6,chunk_0002.mp3,non_fraud,Account maintenance — update address/email,Genuine signals: Offers multiple identifiers; ...,/content/chunks/id_6/id_6_chunk_0002.mp3,...,train,0,"([0.5], 0.5, \n0.5\n)","([0.5], 0.5, 0.5 )",0,2,2,2,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,test,\n\nPast Conversation : You're speaking with K...,test,\n\nPast Conversation : You're speaking with K...,id_18,chunk_0003.mp3,fraud,Transaction fishing — probing for account details,Seeks granular transaction intel; waves card n...,/content/chunks/id_18/id_18_chunk_0003.mp3,...,test,1,"([1.0], 1, 1.0)","([0.75], 0.75, 0.75)",2,1,2,2,5,6
14,test,<think>There are several points to consider:\n...,test,"<think>First things first, Karen provides a re...",id_5,chunk_0001.mp3,non_fraud,Disputes — goods not received,Genuine signals: Provides exact amounts/dates;...,/content/chunks/id_5/id_5_chunk_0001.mp3,...,test,0,"([0.75], 0.25, 0.75)","([0.25], 0.75, 0.25)",2,2,2,2,6,6
15,test,"<think>\nIn this particular case, several red ...",test,Let's go through the conversation to assess th...,id_10,chunk_0001.mp3,non_fraud,Card replacement — lost/damaged,Genuine signals: Immediate freeze and address ...,/content/chunks/id_10/id_10_chunk_0001.mp3,...,test,0,"([0.75], 0.25, 0.75)","([0.25], 0.75, 0.25)",2,0,2,2,4,6
16,test,#### Reasoning:\n1. The customer requests imme...,test,<think>\nThe customer states that they aren't ...,id_16,chunk_0001.mp3,fraud,Override blocks — whitelist merchant/lift frau...,Requests whitelist/bypass; pushes new phone nu...,/content/chunks/id_16/id_16_chunk_0001.mp3,...,test,1,"([0.75], 0.75, 0.75)","([0.75], 0.75, 0.75)",0,2,2,2,6,4


In [ ]:
all_dfs.head()

,data_split,base_completion,peft_base_completion
0,train,\n\nPast Conversation : Thank you for calling ...,NaN
1,train,<think>\nThe context given presents a scenario...,NaN
2,train,"\n\nPast Conversation : American Express, Rene...",NaN
3,train,"\n\nPast Conversation : American Express, this...",NaN
4,train,"\n\nPast Conversation : American Express, Alic...",NaN


In [ ]:
{'model':['base']*len(base_train_completions),'data_split':['train']*len(base_train_completions),'completion':base_train_completions}
{'model':['base']*len(base_test_completions),'data_split':['test']*len(base_test_completions),'completion':base_test_completions}

{'model':['base']*len(base_train_completions),'data_split':['train']*len(base_train_completions),'completion':base_train_completions}
{'model':['peft_base']*len(peft_base_train_completions),'data_split':['train']*len(base_train_completions),'completion':base_train_completions}



Past Conversation : Amex Disputes, this is Karen. Hi Mohit, I'm a calling. I ordered a coffee machine from Beanworks on October 2 for $249.90 and it still hasn't arrived. I'm sorry about that Mohit. Let me verify last four digits, Zepce and email. Zepce 80 Zep 1001 email mohet.vermahudsonmail.com Thank you, Verified. I see the transaction posted October 3. Do you have any tracking or merchant communication? Jack and Shoe's label created a no movement for two weeks. He emailed the merchant three times and got no reply. Understood. I'll open a goods not received case and issue a temporary credit while we request proof of shipment and delivery from the merchant. If they can't provide it, the credit becomes permanent. Sounds fair. If it turns up later, I'll let you know. Thank you. I've created Case Fit 13477. You'll receive an email shortly with what we filed and any documents you can add, screenshots, order confirmation,<think>**Thinking process:**

1. The customer mentions they are ca

In [ ]:
print(train_prompts[0][-1]['content'][-1]['text'])

You are given an audio conversation and past conversation happening between an american express customer care professional and a customer.
This customer can be either a genuine customer or a fraudster.

## Your Task 
Predict a probability that the customer is a fraudster

In order to predict the probability of fraud you should think and analyze the situation and create a reasoning.
Make sure you see each and every aspect of the case and create a case report while reasoning about the case.
Note that the permissible values of probability are 0.0, 0.25, 0.5, 0.75, 1.0

The reasoning process should be enclosed within <think> </think> tags
While the final answer should be enclosed in <answer> </answer> tags

Example Response : 
<think>This is my reasoning</think>
<answer>0.75</answer>

Past Conversation : Thank you for calling American Express. You're speaking with Maya. How can I help today? Hi, Maya. This is Alex Bennett. I'm calling about my September statement. The interest looks a lot 

In [ ]:
print(base_train_completions[0])

You are given an audio conversation and past conversation happening between an american express customer care professional and a customer.
This customer can be either a genuine customer or a fraudster.

## Your Task 
Predict a probability that the customer is a fraudster

In order to predict the probability of fraud you should think and analyze the situation and create a reasoning.
Make sure you see each and every aspect of the case and create a case report while reasoning about the case.
Note that the permissible values of probability are 0.0, 0.25, 0.5, 0.75, 1.0

The reasoning process should be enclosed within <think> </think> tags
While the final answer should be enclosed in <answer> </answer> tags

Example Response : 
<think>This is my reasoning</think>
<answer>0.75</answer>

Past Conversation : Thank you for calling American Express. You're speaking with Maya. How can I help today? Hi, Maya. This is Alex Bennett. I'm calling about my September statement. The interest looks a lot 